# Sentiment-Analyse von TikTok-Kommentaren

Dieses Notebook führt eine Sentiment-Analyse (Stimmungsanalyse) für Kommentare durch, die von einer TikTok-Video-Seite mit einem Selenium-Skript gesammelt und in einer JSON-Datei gespeichert wurden.

**Ziel:** Die vorherrschende Stimmung (positiv, negativ, neutral) in den Kommentaren zu einem bestimmten TikTok-Video zu ermitteln.

**Verwendetes Modell:** Wir nutzen das multilinguale Sentiment-Analyse-Modell `tabularisai/multilingual-sentiment-analysis` von Hugging Face. Dieses Modell kann Kommentare in verschiedenen Sprachen analysieren, ist aber möglicherweise weniger präzise für eine spezifische Sprache als ein dediziertes Modell für diese Sprache.

**Wichtig:** Dieses Notebook dient zu Demonstrations- und Lernzwecken. Die Ergebnisse einer automatisierten Sentiment-Analyse sollten stets mit Vorsicht interpretiert werden, da Kontext, Ironie und Sarkasmus oft schwer zu erfassen sind.

## 1. Vorbereitung: Bibliotheken installieren und JSON-Datei hochladen

Zuerst installieren wir die notwendigen Python-Bibliotheken. Anschließend müssen Sie die JSON-Datei, die von Ihrem TikTok-Scraper generiert wurde (z.B. `tiktok_data_VIDEOID.json`), in das Arbeitsverzeichnis dieses Colab-Notebooks hochladen.

**Anleitung zum Hochladen:**
1. Klicken Sie auf das Ordner-Symbol in der linken Seitenleiste von Colab.
2. Klicken Sie auf das "Hochladen"-Symbol (Pfeil nach oben) und wählen Sie Ihre JSON-Datei aus.

In [ ]:
# Installiere die 'transformers' Bibliothek für die Sentiment-Analyse,
# 'pandas' für die Datenverarbeitung und 'matplotlib' sowie 'seaborn' für Visualisierungen.
!pip install transformers[sentencepiece] pandas matplotlib seaborn

In [ ]:
# Importiere benötigte Bibliotheken
import json
import pandas as pd
from transformers import pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re # Für die Like-Konvertierung

print('Bibliotheken erfolgreich importiert.')

## 2. JSON-Datei laden und Kommentare extrahieren

Geben Sie hier den Namen Ihrer hochgeladenen JSON-Datei an.

In [ ]:
# BITTE DEN DATEINAMEN ANPASSEN, falls Ihre Datei anders heißt!
json_filename = 'tiktok_data_7500353799500926230.json' # Beispielname, bitte an Ihre Datei anpassen

try:
    with open(json_filename, 'r', encoding='utf-8') as f:
        video_data_raw = json.load(f)
    print(f'JSON-Datei "{json_filename}" erfolgreich geladen.')
    
    # Kommentare extrahieren und in einen Pandas DataFrame umwandeln
    if 'comments' in video_data_raw and isinstance(video_data_raw['comments'], list) and video_data_raw['comments']:
        df_comments = pd.DataFrame(video_data_raw['comments'])
        print(f'Anzahl der Kommentare: {len(df_comments)}')
        print(f'Spalten im Kommentar-DataFrame: {df_comments.columns.tolist()}')
        print("\nErste paar Kommentare zur Überprüfung:")
        display(df_comments.head()) # display() für schönere Ausgabe in Colab
    else:
        print("Keine Kommentare in der JSON-Datei gefunden oder die Kommentarliste ist leer.")
        df_comments = pd.DataFrame() # Leerer DataFrame, falls keine Kommentare
except FileNotFoundError:
    print(f"FEHLER: Die Datei '{json_filename}' wurde nicht gefunden. Bitte laden Sie sie hoch und überprüfen Sie den Namen.")
except json.JSONDecodeError:
    print(f"FEHLER: Die Datei '{json_filename}' ist keine valide JSON-Datei. Bitte überprüfen Sie den Inhalt.")
except Exception as e:
    print(f"Ein Fehler ist beim Laden oder Verarbeiten der JSON-Datei aufgetreten: {e}")

## 3. Datenbereinigung und -vorbereitung

Wir bereinigen die Kommentartexte leicht und konvertieren die Like-Zahlen in numerische Werte, falls vorhanden.

In [ ]:
def convert_like_count(like_str):
    """Konvertiert Like-Angaben wie '1.2K' oder '1M' in Zahlen."""
    if isinstance(like_str, (int, float)):
        return int(like_str)
    if pd.isna(like_str) or not isinstance(like_str, str):
        return 0
    
    like_str_cleaned = like_str.lower().strip().replace(',', '.') # Ersetze Komma durch Punkt für Floats
    if 'k' in like_str_cleaned:
        return int(float(like_str_cleaned.replace('k', '')) * 1000)
    elif 'm' in like_str_cleaned:
        return int(float(like_str_cleaned.replace('m', '')) * 1000000)
    elif like_str_cleaned.replace('.', '', 1).isdigit(): # Erlaube einen Punkt für Dezimalzahlen (obwohl bei Likes selten)
        return int(float(like_str_cleaned))
    return 0 # Fallback für unbekannte Formate oder 'N/A'

if 'df_comments' in locals() and not df_comments.empty:
    # Fehlende Texte durch leere Strings ersetzen (wichtig für die Pipeline)
    df_comments['text'] = df_comments['text'].fillna('').astype(str)
    
    # Like-Zahlen konvertieren
    if 'likes' in df_comments.columns:
        df_comments['likes_numeric'] = df_comments['likes'].apply(convert_like_count)
        print("\nLikes konvertiert (Beispiel):")
        display(df_comments[['likes', 'likes_numeric']].head())
    else:
        print("Spalte 'likes' nicht im DataFrame gefunden. Setze 'likes_numeric' auf 0.")
        df_comments['likes_numeric'] = 0
        
    # Entferne Kommentare ohne Text, falls welche durchgerutscht sind
    initial_comment_count = len(df_comments)
    df_comments = df_comments[df_comments['text'].str.strip() != '']
    print(f"Anzahl Kommentare vor Entfernung leerer Texte: {initial_comment_count}")
    print(f"Anzahl Kommentare nach Entfernung leerer Texte: {len(df_comments)}")
else:
    print("Kommentar-DataFrame ist leer oder nicht geladen. Überspringe Datenbereinigung.")

## 4. Initialisierung der Sentiment-Analyse Pipeline

Wir laden das vortrainierte multilinguale Modell. Dies kann beim ersten Mal einige Minuten dauern, da das Modell (ca. 1-2 GB) heruntergeladen wird.

In [ ]:
# Initialisiere die Sentiment-Analyse Pipeline
print('Initialisiere multilinguale Sentiment-Analyse Pipeline... Dies kann einen Moment dauern.')
try:
    sentiment_analyzer = pipeline(
        task='sentiment-analysis', 
        model='tabularisai/multilingual-sentiment-analysis', 
        # device=0 # Für GPU-Nutzung entkommentieren (falls CUDA verfügbar ist)
    )
    print('Sentiment-Analyse Pipeline erfolgreich initialisiert.')
except Exception as e:
    print(f"Fehler bei der Initialisierung der Pipeline: {e}")
    print("Stellen Sie sicher, dass eine Internetverbindung besteht und die 'transformers'-Bibliothek korrekt installiert ist.")
    sentiment_analyzer = None # Setze auf None, damit spätere Zellen nicht fehlschlagen

## 5. Sentiment-Analyse der Kommentare durchführen

Nun wenden wir das initialisierte Modell auf jeden Kommentartext an. Da dies für viele Kommentare dauern kann, geben wir alle 50 Kommentare eine Statusmeldung aus.

In [ ]:
def analyze_sentiment_for_comment(text_to_analyze):
    """Analysiert den Sentiment eines einzelnen Kommentartextes."""
    if not text_to_analyze or pd.isna(text_to_analyze):
        return {'label': 'NO_TEXT', 'score': 0.0}
    try:
        # Das Modell hat eine maximale Sequenzlänge (oft 512 Tokens).
        # TikTok Kommentare sind meist kurz, aber zur Sicherheit kürzen wir.
        # Die Pipeline sollte dies intern handhaben, aber explizit ist sicherer.
        max_char_length = 400 # Zeichen, nicht Tokens, für eine einfache, sichere Kürzung
        truncated_text = text_to_analyze[:max_char_length]
        
        result = sentiment_analyzer(truncated_text)
        if result and isinstance(result, list):
            # Das Modell 'tabularisai/multilingual-sentiment-analysis' gibt Labels wie 'positive', 'negative', 'neutral' zurück.
            return result[0] 
        else:
            return {'label': 'ERROR_PARSING_RESULT', 'score': 0.0}
    except Exception as e:
        # print(f"Fehler bei der Analyse von Text: '{str(text_to_analyze)[:50]}...': {e}") # Für Debugging
        return {'label': 'ERROR_ANALYSIS', 'score': 0.0}

if 'df_comments' in locals() and not df_comments.empty and 'sentiment_analyzer' in locals() and sentiment_analyzer is not None:
    print(f"Führe Sentiment-Analyse für {len(df_comments)} Kommentare durch...")
    
    sentiment_labels = []
    sentiment_scores = []
    
    start_time_analysis = time.time()
    
    # Iteriere über die Zeilen des DataFrames
    # Für eine schnelle Demo kann man .head(X) verwenden, z.B. df_comments.head(50).iterrows()
    for index, row in df_comments.iterrows(): 
        comment_text = row['text']
        
        if not comment_text.strip():
            sentiment_result = {'label': 'NO_TEXT', 'score': 0.0}
        else:
            sentiment_result = analyze_sentiment_for_comment(comment_text)
        
        sentiment_labels.append(sentiment_result['label'])
        sentiment_scores.append(sentiment_result.get('score', 0.0))
        
        if (index + 1) % 50 == 0:
            elapsed_time = time.time() - start_time_analysis
            print(f"--- {index + 1}/{len(df_comments)} Kommentare verarbeitet in {elapsed_time:.2f} Sekunden ---")

    # Füge die Ergebnisse als neue Spalten zum DataFrame hinzu
    df_comments['sentiment_label'] = sentiment_labels
    df_comments['sentiment_score'] = sentiment_scores
    
    end_time_analysis = time.time()
    print(f'\nSentiment-Analyse für {len(df_comments)} Kommentare abgeschlossen in {(end_time_analysis - start_time_analysis):.2f} Sekunden.')
    
    print("\nDataFrame mit Sentiment-Ergebnissen (erste Zeilen):")
    display(df_comments[['author', 'text', 'likes_numeric', 'sentiment_label', 'sentiment_score']].head())
    
    # Speichere den DataFrame mit den Sentiment-Ergebnissen (optional)
    # Erzeugt einen neuen Dateinamen basierend auf dem Original
    base_name = json_filename.rsplit('.', 1)[0]
    output_filename_csv = f"{base_name}_mit_sentiment.csv"
    output_filename_excel = f"{base_name}_mit_sentiment.xlsx"
    
    try:
        df_comments.to_csv(output_filename_csv, index=False, encoding='utf-8-sig') # utf-8-sig für bessere Excel-Kompatibilität mit Umlauten
        print(f"\nDaten mit Sentiment-Analyse gespeichert in CSV: {output_filename_csv}")
        # df_comments.to_excel(output_filename_excel, index=False) 
        # print(f"Daten mit Sentiment-Analyse gespeichert in Excel: {output_filename_excel}")
    except Exception as e:
        print(f"Fehler beim Speichern der Ergebnisse: {e}")

else:
    print("Voraussetzungen für die Analyse (DataFrame, Pipeline) nicht erfüllt. Analyse wird übersprungen.")

## 6. Auswertung und Visualisierung der Ergebnisse

Nachdem die Sentiment-Analyse durchgeführt wurde, können wir uns die Verteilung der Sentiments ansehen und einige einfache Auswertungen vornehmen.

In [ ]:
if 'df_comments' in locals() and not df_comments.empty and 'sentiment_label' in df_comments.columns:
    print("\n--- Auswertung der Sentiment-Ergebnisse ---")
    
    # 1. Verteilung der Sentiment-Labels
    sentiment_counts = df_comments['sentiment_label'].value_counts()
    sentiment_percentages = df_comments['sentiment_label'].value_counts(normalize=True) * 100
    
    print("\nVerteilung der Sentiment-Labels:")
    print(sentiment_counts)
    print("\nProzentuale Verteilung der Sentiment-Labels:")
    print(sentiment_percentages.round(2).astype(str) + ' %')
    
    # 2. Visualisierung der Sentiment-Verteilung
    plt.figure(figsize=(8, 6))
    sns.countplot(data=df_comments, x='sentiment_label', order=sentiment_counts.index, palette='viridis')
    plt.title('Verteilung der Sentiment-Labels in den Kommentaren', fontsize=15)
    plt.xlabel('Sentiment Label', fontsize=12)
    plt.ylabel('Anzahl der Kommentare', fontsize=12)
    plt.xticks(rotation=0)
    # Füge die Anzahl über den Balken hinzu
    for i, count in enumerate(sentiment_counts):
        plt.text(i, count + (0.01 * len(df_comments)), str(count), ha='center', va='bottom', fontsize=10)
    plt.tight_layout()
    plt.show()
    
    # 3. Durchschnittlicher Sentiment-Score pro Label (falls Scores aussagekräftig sind)
    #    Das Modell 'tabularisai/multilingual-sentiment-analysis' gibt diskrete Labels aus (positive, negative, neutral).
    #    Der Score ist die Konfidenz des Modells für dieses Label. 
    #    Eine Aggregation der Scores ist hier weniger sinnvoll als bei Modellen, die einen kontinuierlichen Score von -1 bis 1 liefern.
    #    Wir können aber die durchschnittliche Konfidenz pro Label anzeigen.
    if 'sentiment_score' in df_comments.columns:
        avg_confidence = df_comments.groupby('sentiment_label')['sentiment_score'].mean().sort_values(ascending=False)
        print("\nDurchschnittliche Konfidenz (Score) pro Sentiment-Label:")
        print(avg_confidence.round(4))
        
        plt.figure(figsize=(8, 5))
        avg_confidence.plot(kind='bar', color=sns.color_palette('coolwarm', len(avg_confidence)))
        plt.title('Durchschnittliche Konfidenz pro Sentiment-Label', fontsize=15)
        plt.xlabel('Sentiment Label', fontsize=12)
        plt.ylabel('Durchschnittlicher Score (Konfidenz)', fontsize=12)
        plt.xticks(rotation=0)
        plt.ylim(0, 1) # Scores sind typischerweise zwischen 0 und 1
        plt.tight_layout()
        plt.show()
        
    # 4. Beispiele für Kommentare pro Sentiment-Kategorie
    print("\nBeispiel-Kommentare pro Sentiment-Kategorie:")
    for label in sentiment_counts.index:
        print(f"\n--- {label.upper()} Kommentare (bis zu 3 Beispiele) ---")
        sample_comments = df_comments[df_comments['sentiment_label'] == label]['text'].sample(min(3, len(df_comments[df_comments['sentiment_label'] == label]))).tolist()
        for i, comment_text in enumerate(sample_comments):
            print(f"{i+1}. {comment_text[:150]}{'...' if len(comment_text) > 150 else ''}") # Kommentare kürzen

else:
    print("DataFrame oder Sentiment-Spalten nicht vorhanden für die detaillierte Auswertung.")

## 7. Mögliche Weiterführungen und Limitationen

### Weiterführende Analysen:
- **Sentiment im Zeitverlauf:** Wenn Ihre Daten Zeitstempel für Kommentare hätten (z.B. genaue Upload-Zeiten statt nur relative Angaben wie "Vor 2 T."), könnten Sie analysieren, wie sich das Sentiment über die Zeit entwickelt hat.
- **Sentiment in Relation zu Likes:** Untersuchen, ob Kommentare mit bestimmtem Sentiment tendenziell mehr (oder weniger) Likes erhalten. Hierfür müssten die Like-Zahlen der Kommentare zuverlässig erfasst werden.
- **Themenmodellierung (Topic Modeling):** Innerhalb der positiven/negativen Kommentare könnten Sie versuchen, die Hauptthemen zu identifizieren (z.B. mit LDA oder BERTopic).
- **Wortwolken:** Erstellen Sie Wortwolken für jede Sentiment-Kategorie, um häufige Begriffe zu visualisieren.
- **Vergleich zwischen Videos:** Führen Sie diese Analyse für mehrere Videos durch und vergleichen Sie die Sentiment-Verteilungen.

### Limitationen:
- **Genauigkeit des Modells:** Kein Sentiment-Analyse-Modell ist perfekt. Besonders bei subtilen Äußerungen, Ironie, Sarkasmus oder sehr kontextabhängigen Aussagen kann es zu Fehlklassifikationen kommen.
- **Sprachliche Nuancen:** Emojis, Umgangssprache, Dialekte und kulturelle Referenzen können die Analyse erschweren.
- **Mehrdeutigkeit:** Manche Kommentare sind objektiv schwer einzuordnen.
- **Sampling Bias des Scrapings:** Die gescrapten Kommentare repräsentieren möglicherweise nicht alle Kommentare, die jemals zu dem Video gepostet wurden (z.B. wenn nicht alle geladen werden konnten oder einige später gelöscht wurden).
- **'Likes' für Kommentare:** Die im Beispiel-JSON gezeigten "likes: 0" deuten darauf hin, dass die Like-Zahlen für Kommentare im Scraping-Prozess entweder nicht erfasst wurden oder tatsächlich 0 waren. Für eine Analyse der Korrelation zwischen Sentiment und Likes wären valide Like-Zahlen für die Kommentare selbst notwendig.

**Empfehlung:** Betrachten Sie die Ergebnisse als ersten Indikator und ergänzen Sie sie idealerweise durch qualitative Analysen ausgewählter Kommentare, um ein tiefergehendes Verständnis zu gewinnen.